__TODO:__
1. Постройте список уникальных типов самолетов зарегистрированных в России
2. Какой тип самолета имеет самую раннюю дату выдачи сертификата?
3. Постройте запрос: Владелец аэропорта, Аэропорт, Пассажиропоток суммарный за 2018 год, Грузопоток суммарный за 2018 год
4. Перечислите аэропорты, где пассажиропоток меньше медианы, а  грузопоток больше медианы
5. Перечислите авиакомпании у которых нет типов воздушных судов зарегистрированных в России
6. Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт в котором пассажиропоток в данном месяце максимальный
7. Выведите список: Тип аэропорта, средний грузопоток в месяц в аэропортах данного типа

__NOTE:__ Везде где не указан конкретный временной промежуток или другим образом не указано время был взят 2018 год (последний год достпуный в даных)

In [ ]:
import pandas as pd
from itertools import chain

In [ ]:
aircraft_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/aircraft.csv',
                            sep=';', parse_dates = ['дата действующего свидетельства о регистрации'])

In [ ]:
airlines_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/airlines.csv', 
                            names=['Название_короткое', "Название_полное", "Аэропорт", "Флотилия"])

In [ ]:
airports_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/airports.csv', 
                            names=['Аэропорт', 'Сертификат', "Владелец", "Тип"])

In [ ]:
cargo_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/cargo%20transportation.csv', 
                         sep=';', na_values='***', dtype='str')
# Proper way
# cargo = pd.read_csv('cargo transportation.csv', sep=';', thousands = ' ', decimal = ',', na_values = ['***'])

In [ ]:
passenger_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/passenger%20'\
                             'transportation.csv', sep=';', na_values='***', dtype='str') 

---

## T1

Постройте список уникальных типов самолетов зарегистрированных в России

In [ ]:
aircraft_data[(aircraft_data['Вид воздушного судна'] == 'самолет') &
              (aircraft_data['Опознавательные знаки'].str.startswith('RA'))]\
                ['Тип (наименование) воздушного судна'].unique()

## T2

Какой тип самолета имеет самую раннюю дату выдачи сертификата?

In [ ]:
aircraft_data[aircraft_data['дата действующего свидетельства о регистрации'] ==
              aircraft_data['дата действующего свидетельства о регистрации'].min()]\
                ['Тип (наименование) воздушного судна']

## T3

Постройте запрос: Владелец аэропорта, Аэропорт, Пассажиропоток суммарный за 2018 год, Грузопоток суммарный за 2018 год

### airports_data preparation

In [ ]:
airports_data['Наименование аэропорта РФ'] =\
    airports_data['Аэропорт'].astype('str').apply(lambda x: x.strip().split()[0])

### passenger_data preparation

In [ ]:
passenger_data.drop(columns=['Январь - Декабрь'], inplace=True)
passenger_data.fillna(0, inplace=True)

In [ ]:
for column in passenger_data.columns[1:]:
    passenger_data[column] = passenger_data[column].astype('str').apply(lambda x: int(x.replace(' ', '')))

In [ ]:
passenger_data['Суммарный пасажиропоток'] = passenger_data.iloc[:, 2:].sum(axis=1).copy()

### cargo_data preparation

In [ ]:
cargo_data['Год периода данных'] = cargo_data['Год периода данных'].apply(lambda x: int(x.replace(' ', '')))

In [ ]:
cargo_data.drop(columns=['Январь - Декабрь'], inplace=True)
cargo_data.fillna(0, inplace=True)

In [ ]:
for column in cargo_data.columns[2:]:
    cargo_data[column] = cargo_data[column].astype('str').apply(lambda x: float(x.replace(',', '.').replace(' ', '')))

In [ ]:
cargo_data['Суммарный грузопоток'] = cargo_data.iloc[:, 2:].sum(axis=1).copy()

### merge

In [ ]:
p1 = airports_data[['Владелец', 'Наименование аэропорта РФ']].\
    merge(passenger_data[passenger_data['Год периода данных'] == 2018], 
          on='Наименование аэропорта РФ', how='left').iloc[:, [0, 1, -1]].copy()

In [ ]:
t3 = p1.merge(cargo_data[cargo_data['Год периода данных'] == 2018], 
         on='Наименование аэропорта РФ', how='left').iloc[:, [0, 1, 2, -1]].copy()

In [ ]:
t3.drop_duplicates(keep='first', subset=['Наименование аэропорта РФ',
                                          "Суммарный грузопоток", "Суммарный пасажиропоток"], inplace=True)

## T4

Перечислите аэропорты, где пассажиропоток меньше медианы, а  грузопоток больше медианы

In [ ]:
res[(res['Суммарный грузопоток'] > res['Суммарный грузопоток'].median())].head()

In [ ]:
res[(res['Суммарный пасажиропоток'] < res['Суммарный пасажиропоток'].median())]

In [ ]:
res[(res['Суммарный грузопоток'] > res['Суммарный грузопоток'].median()) 
    & (res['Суммарный пасажиропоток'] < res['Суммарный пасажиропоток'].median())]

### T5 

Перечислите авиакомпании у которых нет типов воздушных судов зарегистрированных в России

In [ ]:
aircraft_data.shape[0] == aircraft_data[(aircraft_data['Опознавательные знаки'].str.startswith('RA'))].shape[0]

In [ ]:
def plane_checker(plane_string:str) -> bool:
    flag = True
    plane_list = [plane_name[:-4].strip() for plane_name in plane_string.split(',')]
    for plane in plane_list:
        if plane in aircraft_data['Тип (наименование) воздушного судна'].values:
            return False
        else:
            continue
    return True

In [ ]:
airlines_data['Все самолеты не РФ'] =\
    airlines_data[airlines_data["Флотилия"].notna()]['Флотилия'].astype('str').apply(plane_checker)

In [ ]:
airlines_data[airlines_data['Все самолеты не РФ'] == True]['Название_короткое'].values

## T6

Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт в котором пассажиропоток в данном месяце максимальный

In [ ]:
best_port = []
for column in passenger_data.columns[2:-1]:
    best_port.append(passenger_data.loc[(passenger_data['Год периода данных'] == 2018) &
                       (passenger_data[column] == passenger_data[column].max())]['Наименование аэропорта РФ'].values)

In [ ]:
res = pd.DataFrame(passenger_data[passenger_data['Год периода данных'] == 2018].iloc[:,2:-1].sum(axis=0), 
             columns=['Cуммарный пассажиропоток']).copy()
res['Аэропорт'] = best_port
res

## T7

Выведите список: Тип аэропорта, средний грузопоток в месяц в аэропортах данного типа 

In [ ]:
airports_data['Тип'] = airports_data['Тип'].str.strip().str.replace(' ', '')

In [ ]:
port_dict = {}
for port_type in airports_data['Тип'].unique():
    port_dict[port_type] = airports_data[airports_data['Тип'] == port_type]['Наименование аэропорта РФ'].values

In [ ]:
# Используем датафрейм из T3
# Суммарный грузопоток за год по все аэропортам --> 
# --> Средний грузопоток за год -(делим на кол-во месяцев)-> Средний грузопоток за месяц 

for key in port_dict: 
    port_dict[key] =t3[t3['Наименование аэропорта РФ'].isin(port_dict.get(key))]['Суммарный грузопоток'].mean()/12

In [ ]:
pd.DataFrame.from_dict(port_dict, orient='index', columns=['Средний грузопоток в месяц'])